In [ ]:
B

In [0]:
use catalog users;
use schema kent_marten;

In [ ]:
CREATE TABLE franchises AS (SELECT * FROM databricks_cookies_dataset_2024.sales.franchises);
CREATE TABLE customers AS SELECT * FROM databricks_cookies_dataset_2024.sales.customers;
CREATE TABLE suppliers AS SELECT * FROM databricks_cookies_dataset_2024.sales.suppliers;
CREATE TABLE transactions AS SELECT * FROM databricks_cookies_dataset_2024.sales.transactions;
CREATE TABLE customer_reviews AS SELECT * FROM databricks_cookies_dataset_2024.media.customer_reviews;
CREATE TABLE gold_reviews_chunked AS SELECT * FROM databricks_cookies_dataset_2024.media.gold_reviews_chunked;

CREATE TABLE xom_places AS SELECT * FROM carto_overture_maps_places.carto.place;




In [0]:
select * from customer_reviews;

review,franchiseID,review_date,new_id,orig_review
"Title: A Delightful Cookie Experience at Bakehouse in Tenmonkan, Kagoshima Bakehouse in Tenmonkan, Kagoshima, has truly captured my heart with their exquisite cookie selection. Among their offerings, the Outback Oatmeal and Austin Almond Biscotti stand out as absolute favorites. The Outback Oatmeal cookie, with its perfect balance of oats and sweetness, provides a satisfying crunch in every bite. Meanwhile, the Austin Almond Biscotti, with its delicate almond flavor and perfect texture, pairs beautifully with a warm cup of coffee. The Orchard Oasis and Golden Gate Ginger cookies also deserve praise. Orchard Oasis is a delightful burst of fruity flavors, while Golden Gate Ginger offers a pleasant kick of spice that warms the palate. I was also pleasantly surprised by the Tokyo Tidbits, which are perfect for those who enjoy a lighter, crispier cookie. Although I didn't try the Pearly Pies, I've heard great things about them from fellow cookie enthusiasts. Bakehouse has certainly made a name for itself in Tenmonkan, and I wholeheartedly recommend a visit for anyone with a sweet tooth. The combination of quality ingredients, unique flavors, and a welcoming atmosphere makes Bakehouse a must-visit destination in Kagoshima.",3000037,2024-05-20T13:24:06.591-04:00,1,"Title: A Delightful Cookie Experience at Bakehouse in Tenmonkan, Kagoshima Bakehouse in Tenmonkan, Kagoshima, has truly captured my heart with their exquisite cookie selection. Among their offerings, the Outback Oatmeal and Austin Almond Biscotti stand out as absolute favorites. The Outback Oatmeal cookie, with its perfect balance of oats and sweetness, provides a satisfying crunch in every bite. Meanwhile, the Austin Almond Biscotti, with its delicate almond flavor and perfect texture, pairs beautifully with a warm cup of coffee. The Orchard Oasis and Golden Gate Ginger cookies also deserve praise. Orchard Oasis is a delightful burst of fruity flavors, while Golden Gate Ginger offers a pleasant kick of spice that warms the palate. I was also pleasantly surprised by the Tokyo Tidbits, which are perfect for those who enjoy a lighter, crispier cookie. Although I didn't try the Pearly Pies, I've heard great things about them from fellow cookie enthusiasts. Bakehouse has certainly made a name for itself in Tenmonkan, and I wholeheartedly recommend a visit for anyone with a sweet tooth. The combination of quality ingredients, unique flavors, and a welcoming atmosphere makes Bakehouse a must-visit destination in Kagoshima."
"""Sweet tooth heaven on East 6th Street! I'm obsessed with Bakehouse's Outback Oatmeal cookies - crunchy, chewy, and perfectly spiced. The Austin Almond Biscotti is another fave, with its subtle nutty flavor and satisfying crunch. However, the Orchard Oasis cookies were a letdown, too sweet and lacking depth. Overall, Bakehouse is a must-visit for cookie lovers in Austin, just be selective with your choices!""",3000017,2024-05-20T13:17:03.052-04:00,2,"""Sweet tooth heaven on East 6th Street! I'm obsessed with Bakehouse's Outback Oatmeal cookies - crunchy, chewy, and perfectly spiced. The Austin Almond Biscotti is another fave, with its subtle nutty flavor and satisfying crunch. However, the Orchard Oasis cookies were a letdown, too sweet and lacking depth. Overall, Bakehouse is a must-visit for cookie lovers in Austin, just be selective with your choices!"""
"**4.5/5 stars** I stumbled upon Bakehouse in the charming Gion district of Kyoto and was thrilled to discover their unique flavor combinations! The Outback Oatmeal cookies were a highlight - perfectly spiced and chewy. The Austin Almond Biscotti was another winner, with a delightful crunch and subtle nutty flavor. However, the Orchard Oasis cookies were a bit too sweet for my taste. The Golden Gate Ginger and Tokyo Tidbits were interesting, but not my favorite. The Pearly Pies, though, were a delightful surprise - flaky crust and a lovely balance of swe

In [0]:
alter table customer_reviews drop column review_sentiment;

In [0]:
BEGIN
  DECLARE vStmtStr STRING;
  FOR columns AS 
    SELECT table_name 
    FROM information_schema.columns
    WHERE table_schema = lower(:schema)
      AND column_name = :columnname
      AND data_type = 'STRING' 
    DO
      SET vStmtStr = 'ALTER TABLE ' || table_name || ' ADD COLUMN (' || :columnname || '_sentiment STRING)';
      EXECUTE IMMEDIATE vStmtStr;
      
      SET vStmtStr = 'MERGE INTO `' || table_name || '` USING (VALUES(1)) ON true WHEN MATCHED THEN UPDATE SET `' || :columnname || '_sentiment` = ai_analyze_sentiment(`' || :columnname || '`)';
      EXECUTE IMMEDIATE vStmtStr;
  END FOR;
END;

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
204,204,0,0


In [0]:
select review_sentiment from customer_reviews;

### David

In [0]:
ALTER TABLE users.kent_marten.customers ADD COLUMN (loyaltyTier STRING);

In [0]:
ALTER TABLE users.kent_marten.transactions SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');
--ALTER TABLE users.kent_marten.customers DROP COLUMN tier;


In [0]:
CREATE OR REPLACE PROCEDURE users.kent_marten.UpdateCustomerLoyaltyTier(
  IN pCustomerID INT
)
LANGUAGE SQL
SQL SECURITY INVOKER
AS
BEGIN
    DECLARE TotalSpend BIGINT;
    DECLARE ReviewCount INT;
    DECLARE Tier STRING;

    -- Calculate total spend
    SET TotalSpend = (
      SELECT SUM(totalPrice)
      FROM users.kent_marten.transactions
      WHERE customerID = pCustomerID
    );

    -- Determine tier
    IF TotalSpend >= 1000
    THEN
      SET Tier = 'Platinum';
    ELSEIF TotalSpend >= 500
    THEN
      SET Tier = 'Gold';
    ELSEIF TotalSpend >= 200
    THEN
      SET Tier = 'Silver';
    ELSE
        SET Tier = 'Bronze';
    END IF;

    -- Update customer record (assumes customers table has loyalty_tier column)
    UPDATE users.kent_marten.customers
    SET loyaltyTier = Tier
    WHERE customerID = pCustomerID;
END


In [0]:
BEGIN
  CALL users.kent_marten.UpdateCustomerLoyaltyTier(1000261);
END

In [0]:
SELECT COUNT(*) FROM users.kent_marten.customers

count(1)
300


In [0]:
SELECT * FROM users.kent_marten.customers WHERE customerID = 1000261

customerID,first_name,last_name,email_address,phone_number,address,city,state,country,continent,postal_zip_code,gender,old_id,loyaltyTier
1000261,Steven,Tanner,haileysanchez@example.net,859-946-4140x24086,08560 Thomas Land,Williamshire,Missouri,Japan,Asia,20642,female,2000261,Bronze


In [0]:
CREATE OR REPLACE PROCEDURE users.kent_marten.UpdateCustomerLoyaltyTiers()
LANGUAGE SQL
SQL SECURITY INVOKER
AS
BEGIN
  FOR customer AS (SELECT DISTINCT customerID FROM users.kent_marten.customers) DO
    CALL users.kent_marten.UpdateCustomerLoyaltyTier(customer.customerID);
  END FOR;
END

In [0]:
CALL users.kent_marten.UpdateCustomerLoyaltyTiers()

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
CREATE OR REPLACE PROCEDURE users.kent_marten.ProductDemandByFranchise(
  IN pFranchiseID INT,
  IN pDate DATE
)
LANGUAGE SQL
SQL SECURITY INVOKER
AS
BEGIN
  DROP TABLE IF EXISTS users.kent_marten.ingredients_count_for_franchise;
  CREATE TABLE users.kent_marten.ingredients_count_for_franchise AS (
    SELECT DATE(dateTime), product, SUM(quantity) AS totalQuantity
    FROM users.kent_marten.transactions
    WHERE franchiseID = pFranchiseID
      AND dateTime > dateadd(day, -30, pDate)
    GROUP BY DATE(dateTime), product
  );

  SELECT * FROM ai_forecast(
    TABLE(users.kent_marten.ingredients_count_for_franchise),
    horizon => dateadd(day, 10, pDate),
    time_col => 'dateTime',
    value_col => 'totalQuantity',
    group_col => 'product'
  )
  ORDER BY product ASC, dateTime ASC;
END

In [0]:
BEGIN
  CALL users.kent_marten.ProductDemandByFranchise(3000047, DATE('2024-05-17'));
END

dateTime,product,totalQuantity_forecast,totalQuantity_upper,totalQuantity_lower
2024-05-17,Austin Almond Biscotti,41.44262295081967,76.51821624572545,0.8154564698723306
2024-05-18,Austin Almond Biscotti,42.049180327868854,78.63205305861732,1.4220134290066966
2024-05-19,Austin Almond Biscotti,42.65573770491804,80.63868709850402,2.0285707446504517
2024-05-20,Austin Almond Biscotti,43.262295081967224,82.53448366520145,2.635128112677158
2024-05-21,Austin Almond Biscotti,43.8688524590164,84.31874621441656,3.241685488400641
2024-05-22,Austin Almond Biscotti,44.47540983606558,85.99327815350247,3.848242865255031
2024-05-23,Austin Almond Biscotti,45.081967213114766,87.56189445101549,4.454800242275596
2024-05-24,Austin Almond Biscotti,45.68852459016395,89.02992943185474,5.0613576193205745
2024-05-25,Austin Almond Biscotti,46.29508196721312,90.4037755142011,5.667914996369127
2024-05-26,Austin Almond Biscotti,46.90163934426231,91.6904754986813,6.27447237341822


In [0]:
SELECT * FROM users.kent_marten.ingredients_count_for_franchise

In [0]:
SELECT * FROM ai_forecast(
  TABLE(users.kent_marten.ingredients_count_for_franchise),
  horizon => dateadd(day, 10, DATE('2024-05-17')),
  time_col => 'dateTime',
  value_col => 'totalQuantity',
  group_col => 'product'
)
ORDER BY product ASC, dateTime ASC;

dateTime,product,totalQuantity_forecast,totalQuantity_upper,totalQuantity_lower
2024-05-17,Austin Almond Biscotti,41.44262295081967,76.51821624572545,0.8154564698723306
2024-05-18,Austin Almond Biscotti,42.049180327868854,78.63205305861732,1.4220134290066966
2024-05-19,Austin Almond Biscotti,42.65573770491804,80.63868709850402,2.0285707446504517
2024-05-20,Austin Almond Biscotti,43.262295081967224,82.53448366520145,2.635128112677158
2024-05-21,Austin Almond Biscotti,43.8688524590164,84.31874621441656,3.241685488400641
2024-05-22,Austin Almond Biscotti,44.47540983606558,85.99327815350247,3.848242865255031
2024-05-23,Austin Almond Biscotti,45.081967213114766,87.56189445101549,4.454800242275596
2024-05-24,Austin Almond Biscotti,45.68852459016395,89.02992943185474,5.0613576193205745
2024-05-25,Austin Almond Biscotti,46.29508196721312,90.4037755142011,5.667914996369127
2024-05-26,Austin Almond Biscotti,46.90163934426231,91.6904754986813,6.27447237341822
